In [ ]:
from CoRe_Dataloader_From_File_With_Random_From_Tensors import get_new_ttv_dataloaders, get_new_test_train_validation_datasets
from CoRe_Dataloader_ECSG import dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader,TensorDataset
import math
import torchinfo
import time
import numpy as np
import wandb
import datetime
from collections import OrderedDict

In [ ]:
import torchmetrics as metrics
import pandas as pd
acc = metrics.Accuracy(task="multiclass", num_classes=19)
combined = metrics.MetricCollection([
    acc,
    metrics.AUROC(task="multiclass", num_classes=19),
    metrics.Precision(task="multiclass", num_classes=19),
    metrics.Recall(task="multiclass", num_classes=19),
    metrics.F1Score(task="multiclass", num_classes=19),
    metrics.FBetaScore(task="multiclass", num_classes=19)
])


def get_df_from_rdict(rdict):
    return pd.DataFrame(pd.Series(rdict).map(lambda x: x.item())).T


In [ ]:
def calc_metrics(model:torch.nn.Module,dl:DataLoader):
    model.eval()
    raw_output = []
    parameters = []
    with torch.no_grad():
        for batch,(sg,params) in enumerate(dl):
            sg = sg.to("cuda:0").to(torch.float)
            sgsh = sg.shape
            sg = sg.view(sgsh[0], 1, sgsh[1], sgsh[2])

            params = params[:,0].to("cuda:0").to(torch.long)
            
            raw_output.append(model(sg).detach().cpu())
            parameters.append(params.cpu())
            print("Validating: ", batch,"/",len(dl),"\t \t \t \t \t \t \t \t \t \t \t \t \t ",end="\r")
    model.train()
    output = torch.vstack(raw_output)
    parameters = torch.hstack(parameters)
    return combined(output,parameters)


In [ ]:
# import vit
# import vit_pytorch
from vit_pytorch import vit_for_small_dataset as vit_sd
from vit_pytorch import vit as simple_vit
from vit_pytorch.deepvit import DeepViT
def init_model():
    # return simple_vit.ViT(image_size=400,
    #                patch_size=20,
    #                num_classes=19,
    #                dim=int(1024/2),
    #                depth=2,
    #                heads=8,
    #                mlp_dim=int(2048/2),
    #                channels=1).to("cuda:0")
    # return vit_sd.ViT(image_size=400,
    #                patch_size=20,
    #                num_classes=19,
    #                dim=1024,
    #                depth=4,
    #                heads=16,
    #                mlp_dim=int(2048/2),
    #                dropout = 0.1,
    #                emb_dropout = 0,
    #                channels=1).to("cuda:0")
    return DeepViT(image_size=400,
                    patch_size=20,
                    num_classes=19,
                    dim=1024,
                    depth=4,
                    heads=16,
                    mlp_dim=int(2048/2),
                    dropout=0.1,
                    emb_dropout=0.1,
                    channels=1).to("cuda:0")


In [ ]:
model = init_model()
img = torch.randn(1,1, 400,400).to("cuda:0")
preds = model(img)  # (1, 1000)
print(preds)
torchinfo.summary(model, input_size=(1,1, 400, 40))

In [ ]:
startlr = 3e-5
optimizer = optim.Adam(params=model.parameters(), lr=startlr)
optimizer1 = optim.NAdam(params=model.parameters(), lr=startlr)
step_scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[1,2,3,4], gamma=0.5)
# at the end of 600 epochs, the learning rate is 0.000,002,62
scheduler = torch.optim.lr_scheduler.StepLR(
    optimizer, step_size=1, gamma=0.8)
scheduler_pl = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max', factor=0.7, patience=35, verbose=True)
lossfn = nn.CrossEntropyLoss()


In [ ]:
def to_seconds(s):
    return f"{s//3600}H:{(s%3600)//60}M:{s%60:5.2}S"
def ismult(n,div):
    return bool(1>>(n%div))

In [ ]:
def train_eval_model(config, train_dl, test_dl, adam=True, nadam=False):
    ldl = len(train_dl)
    pre_acc,max_acc,max_auc = 0,0,0
    results = pd.DataFrame()
    best_model = OrderedDict()
    for epoch in range(1,config.epochs+1):
        
        print("preeval finished")
        etime = time.time()
        for batch,(sg,params) in enumerate(train_dl):
            stime = time.time()
            sgsh = sg.shape
            sg = sg.to("cuda:0").to(torch.float).view(sgsh[0],1,sgsh[1],sgsh[2])
            params = params[:,0].to("cuda:0").to(torch.long)
            optimizer.zero_grad()
            outputs = model(sg)
            loss = lossfn(outputs,params)
            loss.backward()
            optimizer.step() if adam else None
            optimizer1.step() if nadam else None
            #
            torch.cuda.empty_cache()
            #
            wandb.log({"loss": loss.item(), "batch_accuracy": acc(outputs.to(
                "cpu"), params.to("cpu")), "lr": scheduler.get_last_lr()[0], "epoch": epoch})
            
            print(
                f"{epoch:5}/{config.epochs:5} // {batch+1:5}/{ldl:5} | Loss: {loss.item():2.4},batch_accuracy:{acc(outputs.to('cpu'),params.to('cpu')):3.4}, lr:{scheduler.get_last_lr()[0]:1.5}, Time per Batch: {time.time()-stime:.3} seconds, Accumulated Time {to_seconds(round(time.time()-etime,3))}    ", end="\r", flush=True)
            
            if (batch-1) % 5000 == 0:
                print("\nRunning Eval - This will take a WHILE",end = "\r")
                epoch_results = calc_metrics(model, test_dl)
                results = pd.concat([results, get_df_from_rdict(epoch_results)])
                max_acc = max(results["MulticlassAccuracy"])
                max_auc = max(results["MulticlassAUROC"])
                #
                if pre_acc < max_acc:
                    best_model = model.state_dict()
                    try:
                        torch.save(
                            best_model, f"./saved_models/ViT/WithNoise/best_model_state_dict_ViT_for{config.run_name}_stime_{config.start_time.replace(':', '-')}.pt")
                        print("\nSAVING MODEL")
                        with open(f"./saved_models/ViT/WithNoise/best_model_state_dict_ViT_for{config.run_name}_stime_{config.start_time.replace(':', '-')}.txt","a") as file:
                            file.write(f"./saved_models/ViT/WithNoise/best_model_state_dict_ViT_for{config.run_name}_stime_{config.start_time.replace(':', '-')}.pt | {max_acc} {max_auc}\n")
                    except:
                        wandb.alert(level="warning", title="OUT OF MEMORY")

                wandb.log({"epoch": epoch, "lr": scheduler.get_last_lr()[0]} | epoch_results | {
                        "MaximumMulticlassAccuracy": max_acc, "MaximumMulticlassAUROC": max_auc} | {"EpochTime": time.time()-etime})
        #
        scheduler.step()
        step_scheduler.step()
        scheduler_pl.step(max_acc)
        #
        epoch_results = calc_metrics(model, test_dl)
        results = pd.concat([results, get_df_from_rdict(epoch_results)])
        max_acc = max(results["MulticlassAccuracy"])
        max_auc = max(results["MulticlassAUROC"])
        

        wandb.log({"epoch":epoch,"lr":scheduler.get_last_lr()[0]} | epoch_results | {"MaximumMulticlassAccuracy": max_acc, "MaximumMulticlassAUROC":max_auc} | {"EpochTime": time.time()-etime})

    epoch_results = calc_metrics(model, test_dl)
    results = pd.concat([results, get_df_from_rdict(epoch_results)])
    return max_acc,max_auc

In [ ]:
# uncomment for training
results = []
trials = 3
for i in range(trials):
    wandb.init(project="deep_vision_transformer_with_noise_fine_detail_only_classifier",)
    config = wandb.config
    config.run_name = wandb.run._run_id
    config = wandb.config
    config.epochs = 3
    config.inx = 400
    config.iny = 400
    config.lr = startlr     
    config.trial = i+1
    config.total_trials = trials
    config.best_model = OrderedDict()
    config.start_time = datetime.datetime.now().isoformat()
    config.savename = f"best_model_state_dict_at_for{config.run_name}_stime_{config.start_time.replace(':', '-')}__acc_max_acc__auc_auc.pt"
    train_dl, valid_dl, test_dl = get_new_ttv_dataloaders()
    train_eval_model(wandb.config,train_dl,valid_dl,nadam=True)
    results.append(calc_metrics(model,test_dl)) #type: ignore
    if i != (trials - 1):
        model = init_model()

In [ ]:
torch.save(model.state_dict(), f"./best_model_state_dict_ViT_for{config.run_name}_stime_{config.start_time.replace(':', '-')}.pt")

In [ ]:
evaldl = test_dl
model.eval()
raw_output = []
parameters = []
print(len(evaldl))
with torch.no_grad():
    for batch,(sg,params) in enumerate(evaldl):
        sg = sg.to("cuda:0").to(torch.float)
        sgsh = sg.shape
        sg = sg.view(sgsh[0], 1, sgsh[1], sgsh[2])

        params = params[:,0].to("cuda:0").to(torch.long)

        raw_output.append(model(sg).detach().cpu())
        parameters.append(params.cpu())
        print(batch,"finished")
output = torch.vstack(raw_output)
parameters = torch.hstack(parameters)
roc = metrics.ROC(task="multiclass",num_classes=19)
fpr, tpr, thresholds = roc(output,parameters)
len(fpr)
import matplotlib.pyplot as plt
for i in range(len(fpr)):
    plt.plot(fpr[i],tpr[i])
plt.show()
torch.save([fpr,tpr,thresholds],"roc.pt")



In [ ]:
auc = metrics.AUROC(task="multiclass",num_classes=19)
acc = metrics.Accuracy(task="multiclass",num_classes=19)
auc(output,parameters), acc(output,parameters)

In [ ]:
output = torch.argmax(output,dim=1)
output.shape
parameters.shape
comparisons = torch.eq(output,parameters).to(torch.float)
torch.mean(comparisons)
all_params = []
for batch,(sg,params) in enumerate(evaldl):
    all_params.append(params.cpu())
    print(batch,len(evaldl))
all_params = torch.vstack(all_params)
all_params.shape
results = torch.cat((all_params,torch.unsqueeze(comparisons,dim=1)),dim=1)
import pandas as pd
df = pd.DataFrame(results.numpy())
df = df.rename(columns = {0:"EOS",1:"M1",2:"M2",3:"SNR",4:"correct"})
df["combined"] = df["M1"] + df["M2"]
df = df.drop("M1",axis=1).drop("M2", axis=1)
df.rename(columns={"combined":"Masses"})

In [ ]:
by_eos_filter = {}
for i in df["EOS"].unique():
    # print(df[df["EOS"] == i].mean()["correct"])
    by_eos_filter[i] = [df[df["EOS"] == i].mean()["correct"]]

eosdf = pd.DataFrame(by_eos_filter)
eosdf = eosdf.reindex(sorted(eosdf.columns), axis=1)
remapper = {value: key for key, value in dataset.eosmap.items()}
eosdf  = eosdf.rename(columns = remapper)
eosdf


In [ ]:

by_snr_filter = {}
for i in df["SNR"].unique():
    # print(df[df["EOS"] == i].mean()["correct"])
    by_snr_filter[i] = [df[df["SNR"] == i].mean()["correct"]]

snrdf = pd.DataFrame(by_snr_filter)
snrdf = snrdf.reindex(sorted(snrdf.columns), axis=1)
# remapper = {value: key for key, value in dataset.eosmap.items()}
# eosdf.rename(columns=remapper)
# eosdf
snrdf.rename(columns = {0.000:-1}).T
snrdf

In [ ]:

df.to_csv("total_corrects_and_factors.pd.csv")
eosdf.to_csv("accuracy_by_eos.pd.csv")
snrdf.to_csv("accuracy_by_signal-to-noise-ratio.pd.csv")